In [1]:
! [ -e /content ] && pip install -Uqq fastbook kaggle waterfallcharts treeinterpreter dtreeviz

In [2]:
from fastbook import *
from pandas.api.types import is_string_dtype, is_numeric_dtype, is_categorical_dtype
from fastai.tabular.all import *
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor
from IPython.display import Image, display_svg, SVG

pd.options.display.max_rows = 20
pd.options.display.max_columns = 8

/home/tristan/miniconda3/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/tristan/miniconda3/lib/python3.8/site-packages/fastbook/__init__.py:11: UserWarning: Missing `ipywidgets` - please install it
  except ModuleNotFoundError: warn("Missing `ipywidgets` - please install it")
/home/tristan/miniconda3/lib/python3.8/site-packages/fastbook/__init__.py:13: UserWarning: Missing `sentencepiece` - please run `pip install 'sentencepiece<0.1.90'`
  except ModuleNotFoundError: warn("Missing `sentencepiece` - please run `pip install 'sentencepiece<0.1.90'`")


In [3]:
path = Path.cwd()
Path.BASE_PATH = path
path.ls()

(#7) [Path('combined.db-shm'),Path('completed_5m_1h_nonull.csv'),Path('decision_tree.ipynb'),Path('.ipynb_checkpoints'),Path('combined.db'),Path('RF.ipynb'),Path('combined.db-wal')]

In [4]:
df = pd.read_csv(path/'completed_5m_1h_nonull.csv', low_memory=False)
df.columns

Index(['item_name', 'item_id', 'timestamp', 'player_count', 'gst', 'low_p_5m',
       'high_p_5m', 'low_vol_5m', 'high_vol_5m', 'vol_ratio_5m', 'low_p_1h',
       'high_p_1h', 'low_vol_1h', 'high_vol_1h', 'vol_ratio_1h'],
      dtype='object')

In [5]:
dep_var = 'gst'

In [6]:
make_date(df, 'timestamp')
df = add_datepart(df, 'timestamp')
df.columns

/home/tristan/miniconda3/lib/python3.8/site-packages/fastai/tabular/core.py:23: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  df[date_field] = pd.to_datetime(df[date_field], infer_datetime_format=True)


Index(['item_name', 'item_id', 'player_count', 'gst', 'low_p_5m', 'high_p_5m',
       'low_vol_5m', 'high_vol_5m', 'vol_ratio_5m', 'low_p_1h', 'high_p_1h',
       'low_vol_1h', 'high_vol_1h', 'vol_ratio_1h', 'timestampYear',
       'timestampMonth', 'timestampWeek', 'timestampDay', 'timestampDayofweek',
       'timestampDayofyear', 'timestampIs_month_end',
       'timestampIs_month_start', 'timestampIs_quarter_end',
       'timestampIs_quarter_start', 'timestampIs_year_end',
       'timestampIs_year_start', 'timestampElapsed'],
      dtype='object')

In [7]:
procs = [Categorify, FillMissing]

In [8]:
cond = (df.timestampDayofyear<115)
train_idx = np.where(cond)[0]
valid_idx = np.where(~cond)[0]

splits = (list(train_idx),list(valid_idx))

In [9]:
cont,cat = cont_cat_split(df, 1, dep_var=dep_var)

In [10]:
to = TabularPandas(df, procs, cat, cont, y_names=dep_var, splits=splits)

In [11]:
save_pickle(path/'preprocessed_train_valid.pkl', to)

In [12]:
to = load_pickle(path/'preprocessed_train_valid.pkl')

In [13]:
xs,y = to.train.xs,to.train.y
valid_xs,valid_y = to.valid.xs,to.valid.y

In [14]:
def r_mse(pred,y): return round(math.sqrt(((pred-y)**2).mean()), 6)
def m_rmse(m, xs, y): return r_mse(m.predict(xs), y)

In [15]:
m = DecisionTreeRegressor(min_samples_leaf=25)
m.fit(to.train.xs, to.train.y);
m_rmse(m, xs, y), m_rmse(m, valid_xs, valid_y)

(134.765828, 92.449253)